### Importing all the libraries

In [91]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

### Initailising the variables
end_date_for_ind_variable - Indicates the datetime upto which independent variables are <br /> 
campaign_launched - The date on which campaign was launched <br /> 
end_date_tar_variable - Indicates the datetime upto which contributions were accepted <br /> 
campaign_id - Id for selected campaigns <br /> 
number_of_campaigns - Number of campaigns we use for subsetting the data <br /> 


In [92]:
end_date_for_ind_variable = datetime.datetime(2018, 8, 4)
campaign_launched = datetime.datetime(2018, 9, 4)
end_date_tar_variable = datetime.datetime(2019, 6, 18)
campaign_id = 6169
number_of_campaigns = 132

# Reading Data

In [93]:
campaigns = pd.read_csv('./data/campaigns.csv')

donors = pd.read_csv('./data/donors.csv')

gifts = pd.read_csv('./data/gifts.csv')

campaigns_campaign_id = pd.read_csv('./data/selection campaign 6169.csv')

# GIFTS

### Checking data in gifts

In [94]:

print(gifts.isna().sum().sum())
for column in gifts.columns[0:]:
    print(column + " " + str(gifts[column].isna().sum()))

gifts = gifts.dropna()
gifts.info()

15637
Unnamed: 0 0
donorID 0
campaignID 15637
amount 0
date 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200957 entries, 1 to 216593
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  200957 non-null  int64  
 1   donorID     200957 non-null  int64  
 2   campaignID  200957 non-null  float64
 3   amount      200957 non-null  object 
 4   date        200957 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 9.2+ MB


In [95]:
# Dropping any duplicate gifts
gifts.drop_duplicates()

,Unnamed: 0,donorID,campaignID,amount,date
1,1,100001,1577.0,"12,39",2007-10-11
2,2,100001,1634.0,"6,69",2007-12-28
3,3,100001,1778.0,"24,79",2008-06-14
4,4,100001,1808.0,"7,44",2008-07-09
5,5,100001,1933.0,"24,79",2008-12-01
...,...,...,...,...,...
216589,216589,265584,6169.0,"300,0",2018-09-23
216590,216590,281770,6169.0,"30,0",2018-09-08
216591,216591,317880,6169.0,"157,5",2018-09-12
216592,216592,480980,6169.0,"36,0",2018-09-12


### Cleaning data in gifts

In [96]:
gifts['campaignID'] = gifts['campaignID'].fillna(0.0)
gifts['campaignID'] = gifts['campaignID'].astype(int)
gifts['amount'] = gifts['amount'].str.replace(",", ".").astype(float)
gifts['date'] = pd.to_datetime(gifts['date'])
gifts.rename(columns={'Unnamed: 0' : 'giftID'}, inplace=True)
gifts.head()

,giftID,donorID,campaignID,amount,date
1,1,100001,1577,12.39,2007-10-11
2,2,100001,1634,6.69,2007-12-28
3,3,100001,1778,24.79,2008-06-14
4,4,100001,1808,7.44,2008-07-09
5,5,100001,1933,24.79,2008-12-01


In [97]:
gifts['Quater'] = np.where(gifts['date'].dt.month < 4, 'Q1', np.where(gifts['date'].dt.month < 7, 'Q2', np.where(gifts['date'].dt.month < 10, 'Q3','Q4')))
gifts.head()

,giftID,donorID,campaignID,amount,date,Quater
1,1,100001,1577,12.39,2007-10-11,Q4
2,2,100001,1634,6.69,2007-12-28,Q4
3,3,100001,1778,24.79,2008-06-14,Q2
4,4,100001,1808,7.44,2008-07-09,Q3
5,5,100001,1933,24.79,2008-12-01,Q4


In [98]:
gifts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200957 entries, 1 to 216593
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   giftID      200957 non-null  int64         
 1   donorID     200957 non-null  int64         
 2   campaignID  200957 non-null  int32         
 3   amount      200957 non-null  float64       
 4   date        200957 non-null  datetime64[ns]
 5   Quater      200957 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(1)
memory usage: 10.0+ MB


# DONORS

### Checking data in donors

In [99]:
print(donors.isna().sum().sum())
for column in donors.columns[0:]:
    print(column + " " + str(donors[column].isna().sum()))
donors.head()

34
Unnamed: 0 0
donorID 0
zipcode 2
province 0
region 0
gender 32
language 0
dateOfBirth 0


,Unnamed: 0,donorID,zipcode,province,region,gender,language,dateOfBirth
0,0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967
1,1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952
2,2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986
3,3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952
4,4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986


### Cleaning data in donors

In [100]:
donors_zipcode_na = donors.loc[donors['zipcode'].isna(), :]
donors_zipcode_na.head()
donors['zipcode'] = donors['zipcode'].fillna(0)

In [101]:
donors_gender_na = donors.loc[donors['gender'].isna(), :]
donors_gender_na.head()
donors['gender'] = donors['gender'].fillna('Not Known')

In [102]:
donors.drop(['Unnamed: 0'] , axis = 1, inplace=True)
donors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   donorID      44691 non-null  int64  
 1   zipcode      44691 non-null  float64
 2   province     44691 non-null  object 
 3   region       44691 non-null  object 
 4   gender       44691 non-null  object 
 5   language     44691 non-null  object 
 6   dateOfBirth  44691 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.4+ MB


# CAMPAIGNS

### Checking data in campaigns

In [103]:
print(campaigns.isna().sum().sum())
for column in campaigns.columns[0:]:
    print(column + " " + str(campaigns[column].isna().sum()))

campaigns.info()

0
campaignID;date;lettersSent;CostUnit 0
<class 'pandas.core.frame.DataFrame'>
Index: 367 entries, 153;13/12/2004;6 873;0 to 7536;15/12/2019;37 187;0
Data columns (total 1 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   campaignID;date;lettersSent;CostUnit  367 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [104]:
campaigns.head()

,campaignID;date;lettersSent;CostUnit
153;13/12/2004;6 873;0,17 €
154;01/01/2005;7 656;0,30 €
180;18/01/2005;9 933;0,20 €
433;29/01/2005;7 448;0,59 €
476;12/02/2005;6 605;0,51 €


### Cleaning data in campaigns

In [105]:
campaigns = pd.read_csv('./data/campaigns.csv')
campaigns.rename(columns={campaigns.columns[0]: "Cost" }, inplace = True)
campaigns.index.names = ['campaignID;date;lettersSent;CostUnit']
campaigns_reset = campaigns.reset_index()
campaigns_split = campaigns_reset['campaignID;date;lettersSent;CostUnit'].str.split(';', expand=True)
campaigns_split['cost'] = campaigns_reset['Cost']
campaigns_split.rename(columns={0: "campaignID" }, inplace = True)
campaigns_split.rename(columns={1: "date" }, inplace = True)
campaigns_split.rename(columns={2: "lettersSent" }, inplace = True)
campaigns_split.drop([3] , axis = 1, inplace=True)
campaigns_split.head()

,campaignID,date,lettersSent,cost
0,153,13/12/2004,6 873,17 €
1,154,01/01/2005,7 656,30 €
2,180,18/01/2005,9 933,20 €
3,433,29/01/2005,7 448,59 €
4,476,12/02/2005,6 605,51 €


In [106]:
campaigns_split['campaignID'] = campaigns_split['campaignID'].astype(int)
campaigns_split['date'] = pd.to_datetime(campaigns_split['date'])
campaigns_split['lettersSent'] = campaigns_split['lettersSent'].str.replace("\u202f", "").astype(int)
campaigns_split['cost'] = campaigns_split['cost'].str.replace("\xa0€", "").astype(int)
campaigns = campaigns_split
campaigns.head()


,campaignID,date,lettersSent,cost
0,153,2004-12-13,6873,17
1,154,2005-01-01,7656,30
2,180,2005-01-18,9933,20
3,433,2005-01-29,7448,59
4,476,2005-12-02,6605,51


In [107]:
campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   campaignID   367 non-null    int32         
 1   date         367 non-null    datetime64[ns]
 2   lettersSent  367 non-null    int32         
 3   cost         367 non-null    int32         
dtypes: datetime64[ns](1), int32(3)
memory usage: 7.3 KB


## Creating Features

Creating features for gifts tables based on Donor ID

In [108]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_campaignID = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['campaignID'].agg({'number_of_campaigns':'count'})
gifts_donor_total_amount = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['amount'].agg({'total_amount':'sum'})
gifts_donor_last_date = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['date'].agg({'last_date':'max'})
gifts_donor_last_date = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['date'].agg({'last_date':'max'})
gifts_donor_quater_contribution = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID', 'Quater'], as_index=False)['date'].agg({'count'}).unstack('Quater')


donors = donors.merge(gifts_donor_campaignID, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_total_amount, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_last_date, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_quater_contribution, left_on = 'donorID', right_on = 'donorID', how = 'left')

donors['days_since_contributed'] = end_date_for_ind_variable - donors['last_date'] 
donors.head() 

C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,"(count, Q1)","(count, Q2)","(count, Q3)","(count, Q4)",days_since_contributed
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,11.0,166.10,2011-12-30,2.0,3.0,1.0,5.0,2409 days
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,1.0,NaN,NaN,NaN,1302 days
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,NaN,1.0,4.0,NaN,27 days
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,25.0,691.43,2011-06-03,7.0,4.0,4.0,10.0,2619 days
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,NaN,1.0,NaN,3.0,46 days


In [109]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_summary = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID'], as_index=False)['amount'].agg(['mean', 'min', 'max'])
gifts_donor_quater_spending = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID', 'Quater'], as_index=False)['amount'].agg({'mean', 'min', 'max', 'sum'}).unstack('Quater')

donors = donors.merge(gifts_donor_summary, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_quater_spending, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors.head()

C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(min, Q3)","(min, Q4)","(mean, Q1)","(mean, Q2)","(mean, Q3)","(mean, Q4)","(max, Q1)","(max, Q2)","(max, Q3)","(max, Q4)"
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,11.0,166.10,2011-12-30,...,7.44,6.69,15.000000,21.596667,7.44,12.774,20.0,30.0,7.44,24.79
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,NaN,NaN,20.000000,NaN,NaN,NaN,20.0,NaN,NaN,NaN
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,...,3.00,NaN,NaN,4.000000,3.50,NaN,NaN,4.0,5.00,NaN
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,25.0,691.43,2011-06-03,...,24.79,24.79,27.767143,28.697500,24.79,28.311,30.0,30.0,24.79,49.58
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,...,NaN,2.48,NaN,5.000000,NaN,3.320,NaN,5.0,NaN,5.00


In [110]:
#Calculating age for donors till Aug 4th 2018
donors.loc[:,'age'] = donors['dateOfBirth'].apply(lambda x : end_date_for_ind_variable.year - pd.to_datetime(x).year)
donors.loc[:,'age-group'] = donors['age'] // 10 * 10
donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(mean, Q1)","(mean, Q2)","(mean, Q3)","(mean, Q4)","(max, Q1)","(max, Q2)","(max, Q3)","(max, Q4)",age,age-group
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,11.0,166.10,2011-12-30,...,15.000000,21.596667,7.44,12.774,20.0,30.0,7.44,24.79,51,50
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,20.000000,NaN,NaN,NaN,20.0,NaN,NaN,NaN,66,60
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,...,NaN,4.000000,3.50,NaN,NaN,4.0,5.00,NaN,32,30
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,25.0,691.43,2011-06-03,...,27.767143,28.697500,24.79,28.311,30.0,30.0,24.79,49.58,66,60
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,...,NaN,5.000000,NaN,3.320,NaN,5.0,NaN,5.00,32,30


In [111]:
gifts.head()

,giftID,donorID,campaignID,amount,date,Quater
1,1,100001,1577,12.39,2007-10-11,Q4
2,2,100001,1634,6.69,2007-12-28,Q4
3,3,100001,1778,24.79,2008-06-14,Q2
4,4,100001,1808,7.44,2008-07-09,Q3
5,5,100001,1933,24.79,2008-12-01,Q4


Checking for duplicates to ensure Donor IDs are unique after aggregation

In [112]:
campaigns_campaign_id.nunique()

donorID    34888
dtype: int64

In [113]:
selected_donors = donors.merge(campaigns_campaign_id, left_on='donorID', right_on='donorID', how = 'inner')
selected_donors.head()
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34874 entries, 0 to 34873
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   donorID                 34874 non-null  int64          
 1   zipcode                 34874 non-null  float64        
 2   province                34874 non-null  object         
 3   region                  34874 non-null  object         
 4   gender                  34874 non-null  object         
 5   language                34874 non-null  object         
 6   dateOfBirth             34874 non-null  object         
 7   number_of_campaigns     33777 non-null  float64        
 8   total_amount            33777 non-null  float64        
 9   last_date               33777 non-null  datetime64[ns] 
 10  (count, Q1)             18179 non-null  float64        
 11  (count, Q2)             18172 non-null  float64        
 12  (count, Q3)             13483 no

## We are subsetting the data to select top 100 campaigns to which donors have contributed

In [114]:
column = campaigns[campaigns.campaignID == campaign_id].index[0]
row = campaigns[campaigns.campaignID == campaign_id].index[0] - number_of_campaigns
campaigns_selected = campaigns.iloc[row:column+1, :]
campaigns_selected.head()

,campaignID,date,lettersSent,cost
203,3449,2013-04-27,34821,60
204,3505,2013-08-05,32523,21
205,3506,2013-05-26,24470,48
206,3527,2013-06-14,33987,41
207,3528,2013-06-24,32987,13


Calculating the number of campaigns each donor has contributed to

In [115]:
# Checking for mutiple donations for campaign campaign_id
gifts1 = gifts[gifts.campaignID.isin(campaigns_selected.campaignID)]
agg_gifts = gifts1.groupby(['donorID'])['campaignID'].agg(['count']).sort_values(by = 'count', ascending =False)
agg_gifts.head()


,count
donorID,
113149,67
102986,65
123686,60
131428,60
105960,59


In [116]:
gifts1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44564 entries, 12 to 216593
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   giftID      44564 non-null  int64         
 1   donorID     44564 non-null  int64         
 2   campaignID  44564 non-null  int32         
 3   amount      44564 non-null  float64       
 4   date        44564 non-null  datetime64[ns]
 5   Quater      44564 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(1)
memory usage: 3.2+ MB


### Merging the tables to selected donors to initialise the final base table

In [117]:
selected_donors = selected_donors.merge(gifts1, left_on='donorID', right_on='donorID', how = 'inner')
# If Contribution made after May 18 

In [118]:
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21647 entries, 0 to 21646
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   donorID                 21647 non-null  int64          
 1   zipcode                 21647 non-null  float64        
 2   province                21647 non-null  object         
 3   region                  21647 non-null  object         
 4   gender                  21647 non-null  object         
 5   language                21647 non-null  object         
 6   dateOfBirth             21647 non-null  object         
 7   number_of_campaigns     21628 non-null  float64        
 8   total_amount            21628 non-null  float64        
 9   last_date               21628 non-null  datetime64[ns] 
 10  (count, Q1)             17038 non-null  float64        
 11  (count, Q2)             14174 non-null  float64        
 12  (count, Q3)             11590 no

Setting the target variable

In [119]:
selected_donors['has_contributed'] = np.where((selected_donors['campaignID'] == campaign_id) & (selected_donors['amount'] >= 30) & (selected_donors['date'] > campaign_launched) & (selected_donors['date'] < end_date_tar_variable),1, 0)

In [120]:
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21647 entries, 0 to 21646
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   donorID                 21647 non-null  int64          
 1   zipcode                 21647 non-null  float64        
 2   province                21647 non-null  object         
 3   region                  21647 non-null  object         
 4   gender                  21647 non-null  object         
 5   language                21647 non-null  object         
 6   dateOfBirth             21647 non-null  object         
 7   number_of_campaigns     21628 non-null  float64        
 8   total_amount            21628 non-null  float64        
 9   last_date               21628 non-null  datetime64[ns] 
 10  (count, Q1)             17038 non-null  float64        
 11  (count, Q2)             14174 non-null  float64        
 12  (count, Q3)             11590 no

Dropping columns from base table that is not required

In [121]:
drop_columns = ['zipcode', 'dateOfBirth', 'last_date', 'giftID', 'campaignID', 'Quater', 'date']
selected_donors = selected_donors.drop(drop_columns, axis = 1)


Renaming all aggregated columns 

In [122]:
selected_donors = selected_donors.rename(columns={('count', 'Q1') : 'Q1_Donations', ('count', 'Q2') : 'Q2_Donations', ('count', 'Q3') : 'Q3_Donations', ('count', 'Q4') : 'Q4_Donations'})


In [123]:
selected_donors = selected_donors.rename(columns={('mean', 'Q1') : 'Q1_Mean', ('mean', 'Q2') : 'Q2_Mean', ('mean', 'Q3') : 'Q3_Mean', ('mean', 'Q4') : 'Q4_Mean'})

In [124]:
selected_donors = selected_donors.rename(columns={('min', 'Q1') : 'Q1_Min', ('min', 'Q2') : 'Q2_Min', ('min', 'Q3') : 'Q3_Min', ('min', 'Q4') : 'Q4_Min'})

In [125]:
selected_donors = selected_donors.rename(columns={('max', 'Q1') : 'Q1_Max', ('max', 'Q2') : 'Q2_Max', ('max', 'Q3') : 'Q3_Max', ('max', 'Q4') : 'Q4_Max'})

In [126]:
selected_donors = selected_donors.rename(columns={('sum', 'Q1') : 'Q1_Amount', ('sum', 'Q2') : 'Q2_Amount', ('sum', 'Q3') : 'Q3_Amount', ('sum', 'Q4') : 'Q4_Amount'})

Converting the dt to days 

In [127]:
selected_donors['days_since_contributed'] = selected_donors['days_since_contributed'].dt.days

In [128]:
selected_donors.loc[selected_donors.has_contributed == 1, 'has_contributed'].sum()

950

In [129]:
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21647 entries, 0 to 21646
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   donorID                 21647 non-null  int64  
 1   province                21647 non-null  object 
 2   region                  21647 non-null  object 
 3   gender                  21647 non-null  object 
 4   language                21647 non-null  object 
 5   number_of_campaigns     21628 non-null  float64
 6   total_amount            21628 non-null  float64
 7   Q1_Donations            17038 non-null  float64
 8   Q2_Donations            14174 non-null  float64
 9   Q3_Donations            11590 non-null  float64
 10  Q4_Donations            13356 non-null  float64
 11  days_since_contributed  21628 non-null  float64
 12  mean                    21628 non-null  float64
 13  min                     21628 non-null  float64
 14  max                     21628 non-null

Checking the number of donors for 6169 campaigns

In [130]:
selected_donors.loc[selected_donors.has_contributed == 1, 'has_contributed'].count()

950

Converting the numerical values to Scaler

In [131]:
scaler = MinMaxScaler()

for column in selected_donors.columns[1:]:
    df = selected_donors[column]

    if selected_donors[column].dtype != object and selected_donors[column].dtype.name != 'category':
        selected_donors[column] = scaler.fit_transform(np.array(selected_donors[[column]]))

In [132]:
selected_donors.head()

,donorID,province,region,gender,language,number_of_campaigns,total_amount,Q1_Donations,Q2_Donations,Q3_Donations,...,Q3_Mean,Q4_Mean,Q1_Max,Q2_Max,Q3_Max,Q4_Max,age,age-group,amount,has_contributed
0,100002,Antwerp,Flanders,M,NL,0.0,0.001817,0.0,NaN,NaN,...,NaN,NaN,0.007802,NaN,NaN,NaN,0.582278,0.571429,0.013842,0.0
1,100010,Brussels,Brussels,M,FR,0.0,0.003760,0.0,NaN,NaN,...,NaN,NaN,0.015803,NaN,NaN,NaN,0.189873,0.142857,0.027824,0.0
2,100011,Brussels,Brussels,M,NL,0.0,0.009590,NaN,NaN,NaN,...,NaN,0.099820,NaN,NaN,NaN,0.092854,0.443038,0.428571,0.069770,0.0
3,100012,Brussels,Brussels,F,FR,0.0,0.000360,NaN,NaN,NaN,...,NaN,0.004801,NaN,NaN,NaN,0.004466,0.265823,0.285714,0.003356,0.0
4,100013,Brussels,Brussels,F,FR,0.0,0.002789,NaN,NaN,NaN,...,NaN,0.029806,NaN,NaN,NaN,0.027726,0.303797,0.285714,0.020833,0.0


In [133]:
encoder = LabelEncoder()
for column in selected_donors.columns[1:]:
    df = selected_donors[column]

    if selected_donors[column].dtype == object or selected_donors[column].dtype.name == 'category':
        selected_donors[column] = encoder.fit_transform(selected_donors[column])

In [134]:
selected_donors.to_csv(os.path.join('./data/tables',r'processed_table_6169.csv'))